# 使用gensim训练word2vec

本DEMO只使用部分数据，使用全部数据预训练的词向量地址：  

链接: https://pan.baidu.com/s/1ewlck3zwXVQuAzraZ26Euw 提取码: qbpr 

In [12]:
import logging
import random

import numpy as np
import torch


In [13]:
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

# set seed
seed = 666
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [14]:
# split data to 10 fold
fold_num = 10
data_file = 'data/train_set.csv'
import pandas as pd


def all_data2fold(fold_num, num=10000):
    fold_data = []
    f = pd.read_csv(data_file, sep='\t', encoding='UTF-8')
    texts = f['text'].tolist()[:num]
    labels = f['label'].tolist()[:num]

    total = len(labels)

    index = list(range(total))
    np.random.shuffle(index)

    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(texts[i])
        all_labels.append(labels[i])

    label2id = {}
    for i in range(total):
        label = str(all_labels[i])
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)

    all_index = [[] for _ in range(fold_num)]
    for label, data in label2id.items():
        print(label, len(data))
        batch_size = int(len(data) / fold_num)
        other = len(data) - batch_size * fold_num
        for i in range(fold_num):
            cur_batch_size = batch_size + 1 if i < other else batch_size
            print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            all_index[i].extend(batch_data)

    batch_size = int(total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        num = len(all_index[fold])
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]

        if num > batch_size:
            fold_texts = texts[:batch_size]
            other_texts.extend(texts[batch_size:])
            fold_labels = labels[:batch_size]
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
        elif num < batch_size:
            end = start + batch_size - num
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            start = end
        else:
            fold_texts = texts
            fold_labels = labels

        assert batch_size == len(fold_labels)

        # shuffle
        index = list(range(batch_size))
        np.random.shuffle(index)

        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])

        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        fold_data.append(data)

    logging.info("Fold lens %s", str([len(data['label']) for data in fold_data]))

    return fold_data


fold_data = all_data2fold(10)

2020-07-31 22:32:01,859 INFO: Fold lens [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


6 505
51
51
51
51
51
50
50
50
50
50
0 1885
189
189
189
189
189
188
188
188
188
188
7 426
43
43
43
43
43
43
42
42
42
42
2 1561
157
156
156
156
156
156
156
156
156
156
11 164
17
17
17
17
16
16
16
16
16
16
1 1866
187
187
187
187
187
187
186
186
186
186
3 1078
108
108
108
108
108
108
108
108
107
107
4 788
79
79
79
79
79
79
79
79
78
78
9 310
31
31
31
31
31
31
31
31
31
31
5 619
62
62
62
62
62
62
62
62
62
61
8 415
42
42
42
42
42
41
41
41
41
41
12 88
9
9
9
9
9
9
9
9
8
8
10 242
25
25
24
24
24
24
24
24
24
24
13 53
6
6
6
5
5
5
5
5
5
5


In [15]:
# build train data for word2vec
fold_id = 9

train_texts = []
for i in range(0, fold_id):
    data = fold_data[i]
    train_texts.extend(data['text'])
    
logging.info('Total %d docs.' % len(train_texts))

2020-07-31 22:32:03,398 INFO: Total 9000 docs.


In [16]:
!pip install gensim

In [18]:
logging.info('Start training...')
from gensim.models.word2vec import Word2Vec

num_features = 100     # Word vector dimensionality
num_workers = 8       # Number of threads to run in parallel

train_texts = list(map(lambda x: list(x.split()), train_texts))
model = Word2Vec(train_texts, workers=num_workers, size=num_features)
model.init_sims(replace=True)

# save model
model.save("word2vec_7.bin")

2020-07-31 22:33:05,846 INFO: Start training...
2020-07-31 22:33:09,397 INFO: collecting all words and their counts
2020-07-31 22:33:09,433 INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-31 22:33:12,283 INFO: collected 5295 word types from a corpus of 8191447 raw words and 9000 sentences
2020-07-31 22:33:12,303 INFO: Loading a fresh vocabulary
2020-07-31 22:33:13,495 INFO: effective_min_count=5 retains 4335 unique words (81% of original 5295, drops 960)
2020-07-31 22:33:13,496 INFO: effective_min_count=5 leaves 8189498 word corpus (99% of original 8191447, drops 1949)
2020-07-31 22:33:13,532 INFO: deleting the raw counts dictionary of 5295 items
2020-07-31 22:33:13,534 INFO: sample=0.001 downsamples 61 most-common words
2020-07-31 22:33:13,535 INFO: downsampling leaves estimated 7070438 word corpus (86.3% of prior 8189498)
2020-07-31 22:33:13,552 INFO: estimated required memory for 4335 words and 100 dimensions: 5635500 bytes
2020-07-31 22:33:13,554 INF

2020-07-31 22:33:52,637 INFO: EPOCH 5 - PROGRESS: at 39.56% examples, 921384 words/s, in_qsize 15, out_qsize 0
2020-07-31 22:33:53,650 INFO: EPOCH 5 - PROGRESS: at 52.92% examples, 914639 words/s, in_qsize 14, out_qsize 1
2020-07-31 22:33:54,660 INFO: EPOCH 5 - PROGRESS: at 63.73% examples, 886503 words/s, in_qsize 13, out_qsize 2
2020-07-31 22:33:55,670 INFO: EPOCH 5 - PROGRESS: at 74.80% examples, 870323 words/s, in_qsize 12, out_qsize 3
2020-07-31 22:33:56,674 INFO: EPOCH 5 - PROGRESS: at 86.39% examples, 860421 words/s, in_qsize 16, out_qsize 2
2020-07-31 22:33:57,680 INFO: EPOCH 5 - PROGRESS: at 96.08% examples, 838282 words/s, in_qsize 16, out_qsize 0
2020-07-31 22:33:57,954 INFO: worker thread finished; awaiting finish of 7 more threads
2020-07-31 22:33:57,959 INFO: worker thread finished; awaiting finish of 6 more threads
2020-07-31 22:33:57,961 INFO: worker thread finished; awaiting finish of 5 more threads
2020-07-31 22:33:57,974 INFO: worker thread finished; awaiting finish 

In [19]:
# load model
model = Word2Vec.load("word2vec_726.bin")

# convert format
model.wv.save_word2vec_format('word2vec.txt', binary=False)

2020-07-31 22:33:58,163 INFO: loading Word2Vec object from word2vec_726.bin
2020-07-31 22:33:58,219 INFO: loading wv recursively from word2vec_726.bin.wv.* with mmap=None
2020-07-31 22:33:58,221 INFO: setting ignored attribute vectors_norm to None
2020-07-31 22:33:58,223 INFO: loading vocabulary recursively from word2vec_726.bin.vocabulary.* with mmap=None
2020-07-31 22:33:58,224 INFO: loading trainables recursively from word2vec_726.bin.trainables.* with mmap=None
2020-07-31 22:33:58,225 INFO: setting ignored attribute cum_table to None
2020-07-31 22:33:58,227 INFO: loaded word2vec_726.bin
2020-07-31 22:33:58,245 INFO: storing 4327x100 projection weights into word2vec.txt


**关于Datawhale：**

> Datawhale是一个专注于数据科学与AI领域的开源组织，汇集了众多领域院校和知名企业的优秀学习者，聚合了一群有开源精神和探索精神的团队成员。Datawhale 以“for the learner，和学习者一起成长”为愿景，鼓励真实地展现自我、开放包容、互信互助、敢于试错和勇于担当。同时 Datawhale 用开源的理念去探索开源内容、开源学习和开源方案，赋能人才培养，助力人才成长，建立起人与人，人与知识，人与企业和人与未来的联结。

本次新闻文本分类学习，专题知识将在天池分享，详情可关注Datawhale：

 ![](http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/1095279172547/1584432602983_kAxAvgQpG2.jpg)